In [1]:
!wget https://lodmedia.hb.bizmrg.com/case_files/837418/train_dataset_train.zip
!wget https://lodmedia.hb.bizmrg.com/case_files/837418/test_dataset_test.zip
!unzip test_dataset_test.zip
!unzip train_dataset_train.zip

--2022-10-08 23:26:52--  https://lodmedia.hb.bizmrg.com/case_files/837418/train_dataset_train.zip
Resolving lodmedia.hb.bizmrg.com (lodmedia.hb.bizmrg.com)... 217.69.142.100
Connecting to lodmedia.hb.bizmrg.com (lodmedia.hb.bizmrg.com)|217.69.142.100|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 441897856 (421M) [application/zip]
Saving to: ‘train_dataset_train.zip’

train_dataset_train 100%[===================>] 421.43M   411KB/s    in 18m 29s 

2022-10-08 23:45:23 (389 KB/s) - ‘train_dataset_train.zip’ saved [441897856/441897856]

--2022-10-08 23:45:23--  https://lodmedia.hb.bizmrg.com/case_files/837418/test_dataset_test.zip
Resolving lodmedia.hb.bizmrg.com (lodmedia.hb.bizmrg.com)... 217.69.142.100
Connecting to lodmedia.hb.bizmrg.com (lodmedia.hb.bizmrg.com)|217.69.142.100|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 191817805 (183M) [application/zip]
Saving to: ‘test_dataset_test.zip’

test_dataset_test.z 100%[==============

In [2]:
import time
import os
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import dbscan,DBSCAN
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDClassifier, SGDRegressor, RidgeClassifier, LogisticRegressionCV,Ridge,QuantileRegressor,PassiveAggressiveClassifier,PoissonRegressor
from sklearn.ensemble import ExtraTreesRegressor,ExtraTreesClassifier,RandomForestClassifier,VotingClassifier,RandomForestRegressor,GradientBoostingClassifier,GradientBoostingRegressor,StackingRegressor,BaggingClassifier
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.svm import LinearSVC,LinearSVR,SVR,NuSVR
from sklearn.decomposition import TruncatedSVD,PCA,FactorAnalysis,IncrementalPCA,FastICA,KernelPCA,NMF
from sklearn.preprocessing import RobustScaler,QuantileTransformer,PowerTransformer,PolynomialFeatures,KBinsDiscretizer,StandardScaler,OneHotEncoder,OrdinalEncoder,FunctionTransformer,MaxAbsScaler,MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline,FeatureUnion,TransformerMixin
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor,LocalOutlierFactor
from sklearn.model_selection import train_test_split,ShuffleSplit,StratifiedShuffleSplit,TimeSeriesSplit,GridSearchCV,KFold
from sklearn.feature_selection import SelectFromModel
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer,SimpleImputer
from sklearn.dummy import DummyRegressor,DummyClassifier
from sklearn import set_config
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error,roc_auc_score,accuracy_score,f1_score,classification_report,recall_score,make_scorer

In [3]:
train=pd.read_csv('/content/train.csv')
test=pd.read_csv('/content/test.csv')

In [4]:
pipe=Pipeline([('text',CountVectorizer(ngram_range=(4, 12), analyzer="char_wb", max_df=0.5,min_df=2)),
               ('tfidf',TfidfTransformer()),
               ('rgs', KNeighborsRegressor(n_neighbors=4, weights="distance", 
                                           metric="cosine"))])
knn=KNeighborsRegressor(n_neighbors=1, weights="distance", 
                                           metric="manhattan")

In [5]:
test['object_img']=pipe.fit(train['description'],train['object_img']).predict(test['description'])

In [6]:
test['object_img']=knn.fit(pd.DataFrame(pd.Series(os.listdir('/content/test')).str.replace(r'\..+','').astype('int')),
        pd.Series(os.listdir('/content/test')).str.replace(r'\..+','').astype('int')).predict(test[['object_img']])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but KNeighborsRegressor was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [7]:
test['object_img']=test['object_img'].astype('int')

In [8]:
submit=test[['id','object_img']]
submit.to_csv('submitl.csv',index=False)